In [1]:
import os
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader, Subset
from torchvision.datasets import ImageFolder
import torch.optim as optim
import torch.nn as nn
import numpy as np
import pandas as pd

In [2]:
# 设置数据路径
data_dir = 'D:\FDU\Course\DeepLearning\midterm\CUB_200_2011\CUB_200_2011'
images_dir = os.path.join(data_dir, 'images')

# 读取划分文件
split_data = pd.read_csv(os.path.join(data_dir, 'train_test_split.txt'), 
                         delim_whitespace=True, header=None, names=['id', 'train_flag'])

# 设置图像变换
transformations = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 加载全部图像
full_dataset = ImageFolder(root=images_dir, transform=transformations)

# 分割训练集和测试集
train_idx = split_data[split_data['train_flag'] == 1]['id'].values - 1
test_idx = split_data[split_data['train_flag'] == 0]['id'].values - 1

train_dataset = Subset(full_dataset, train_idx)
test_dataset = Subset(full_dataset, test_idx)

In [3]:
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

In [4]:
def train(model, loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    return running_loss / len(loader.dataset)

def validate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == labels.data)
            running_loss += loss.item() * inputs.size(0)
    accuracy = correct.double() / len(loader.dataset)
    return running_loss / len(loader.dataset), accuracy.item()


In [5]:

# 加载预训练的ResNet-18模型
model = resnet18(pretrained=False)

# 修改输出层
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 200)


# 定义设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 为新的输出层设置高学习率，其他层设置低学习率
params_to_update = []
params_names = []

# 只对非全连接层参数应用低学习率
for name, param in model.named_parameters():
    if param.requires_grad == True:
        if name not in ['fc.weight', 'fc.bias']:
            params_to_update.append(param)
            params_names.append(name)


# 初始化TensorBoard
writer = SummaryWriter('runs/birds_experiment_')

# 使用两个参数组
optimizer = optim.SGD([
    {'params': model.fc.parameters(), 'lr': 0.01},  # 高学习率仅适用于全连接层
    {'params': params_to_update, 'lr': 0.001}       # 低学习率适用于其余层
], momentum=0.9)


# 损失函数
criterion = nn.CrossEntropyLoss()

epochs = 20
for epoch in range(epochs):
    train_loss = train(model, train_loader, optimizer, criterion, device)
    test_loss, test_accuracy = validate(model, test_loader, criterion, device)
    print(f'Epoch {epoch+1}, Train Loss: {train_loss}, Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')
    writer.add_scalar('Loss/Train', train_loss, epoch)
    writer.add_scalar('Loss/Test', test_loss, epoch)
    writer.add_scalar('Accuracy/Test', test_accuracy, epoch)

writer.close()


Epoch 1, Train Loss: 5.507534643312594, Test Loss: 5.403356584549608, Test Accuracy: 0.013462202278218847
Epoch 2, Train Loss: 5.197170884441367, Test Loss: 5.068711027343561, Test Accuracy: 0.02744218156713842
Epoch 3, Train Loss: 4.876232302503105, Test Loss: 4.886903369101648, Test Accuracy: 0.03296513634794615
Epoch 4, Train Loss: 4.625261890359191, Test Loss: 4.7269451504622735, Test Accuracy: 0.05298584742837418
Epoch 5, Train Loss: 4.4285221370331715, Test Loss: 4.665962403586786, Test Accuracy: 0.057128063513979976
Epoch 6, Train Loss: 4.275384864450734, Test Loss: 4.43161507709544, Test Accuracy: 0.06506731101139109
Epoch 7, Train Loss: 4.095021944345137, Test Loss: 4.431640689028186, Test Accuracy: 0.07783914394200897
Epoch 8, Train Loss: 3.9226418103143934, Test Loss: 4.4770332278158484, Test Accuracy: 0.0700724887814981
Epoch 9, Train Loss: 3.769778832301005, Test Loss: 4.136719866131436, Test Accuracy: 0.10372799447704521
Epoch 10, Train Loss: 3.5712448138573345, Test Loss